<h1 align="center">Estructura de Datos y Algoritmos II</h1>
<h1 align="center">Evaluación 03 - Componente Práctico</h1>
<h1 align="center">Ejercicio OpenMP: Cálculo de Integrales</h1>
<h1 align="center">2024</h1>
<h1 align="center">MEDELLÍN - COLOMBIA </h1>

# **Cálculo de Integrales por los Métodos de Simpson en Paralelo usando OpenMP**

Este ejemplo tiene como objetivo explicar el **método de Simpson 1/3** para el cálculo de integrales definidas, desarrollando el concepto teórico, un algoritmo en pseudocódigo de la versión secuencial, y finalmente la versión paralela utilizando **OpenMP**.

## **1. Aspectos Teóricos del Método de Simpson 1/3**

El **método de Simpson 1/3** es una técnica numérica para aproximar integrales definidas de funciones continuas. Se basa en la idea de aproximar el área bajo la curva de una función mediante el uso de parábolas, en lugar de rectángulos (como en la regla del trapecio). El método se aplica dividiendo el intervalo de integración en un número par de subintervalos y ajustando una parábola a cada tres puntos consecutivos.

**Fórmula del Método de Simpson 1/3**

Dada una función $ f(x) $, la integral definida:

$$
I = \int_{a}^{b} f(x) \, dx
$$

se aproxima mediante el método de Simpson 1/3 con la siguiente fórmula:

$$
I \approx \frac{h}{3} \left[ f(a) + 4 \sum_{i=1, \text{impar}}^{n-1} f(x_i) + 2 \sum_{i=2, \text{par}}^{n-2} f(x_i) + f(b) \right]
$$

donde:
- $ a $ y $ b $ son los límites de integración.
- $ n $ es el número de subintervalos (debe ser par).
- $ h = \frac{b - a}{n} $ es el ancho de cada subintervalo.
- $ x_i = a + i \cdot h $ es el punto de evaluación dentro del intervalo.

### **Conceptos Importantes**:
1. Los puntos de la función $ f(x) $ se evalúan en $ a $ (límite inferior) y $ b $ (límite superior), así como en puntos impares y pares dentro del intervalo.
2. Los términos impares (multiplicados por 4) son los puntos donde la parábola alcanza su máxima altura.
3. Los términos pares (multiplicados por 2) corresponden a los puntos intermedios entre las parábolas.


## **2. Desarrollo del Algoritmo en Pseudocódigo (Versión Secuencial)**

**Pseudocódigo Secuencial del Método de Simpson 1/3**

```plaintext
Entrada: a (límite inferior), b (límite superior), n (número de subintervalos, debe ser par)
Salida: Aprox. de la integral I

1. Calcular el tamaño del subintervalo:
   h = (b - a) / n

2. Inicializar las sumas:
   sum_odd = 0.0  // Suma de los términos impares
   sum_even = 0.0 // Suma de los términos pares

3. Calcular los términos impares (índices 1, 3, 5,...):
   Para i = 1 hasta n-1 con incremento de 2:
       sum_odd += f(a + i * h)

4. Calcular los términos pares (índices 2, 4, 6,...):
   Para i = 2 hasta n-2 con incremento de 2:
       sum_even += f(a + i * h)

5. Aplicar la fórmula de Simpson:
   I = (h / 3) * (f(a) + 4 * sum_odd + 2 * sum_even + f(b))

6. Retornar I
```

**Descripción del Algoritmo Secuencial:**
1. El intervalo $ [a, b] $ se divide en $ n $ subintervalos de igual tamaño $ h $.
2. Se calculan por separado las sumas de los términos impares y pares.
3. Finalmente, se aplica la fórmula de Simpson para obtener la aproximación de la integral.

**Código en C**

In [ ]:
#include <stdio.h>
#include <math.h>
#include <omp.h>
//Definir la función que queremos integrar
double f(double x) {
    return exp(-x * x);  // f(x) = e^(-x^2)
}

//Implementación secuencial del método de Simpson 1/3
double simpson_sequential(double a, double b, int n) {
    double h = (b - a) / n;
    double sum_odd = 0.0, sum_even = 0.0, result = 0.0;

    // Calcular términos impares
    for (int i = 1; i <= n - 1; i += 2) {
        sum_odd += f(a + i * h);
    }

    // Calcular términos pares
    for (int i = 2; i <= n - 2; i += 2) {
        sum_even += f(a + i * h);
    }

    // Aplicar la fórmula de Simpson
    result = (h / 3) * (f(a) + 4 * sum_odd + 2 * sum_even + f(b));

    return result;
}

int main() {
    double a = -1.0;  // Límite inferior
    double b = 1.0;  // Límite superior
    int n = 1000000; // Número de subintervalos (debe ser par)

    // Medir el tiempo de ejecución
    double start_time = omp_get_wtime();

    double result = simpson_sequential(a, b, n);

    double end_time = omp_get_wtime();

    // Imprimir el resultado y el tiempo de ejecución
    printf("Resultado de la integral (secuencial): %.10f\n", result);
    printf("Tiempo de ejecución (secuencial): %.6f segundos\n", end_time - start_time);

    return 0;
}

SyntaxError: invalid syntax (<ipython-input-2-289c3cd23880>, line 3)

## **3. Identificación de Bloques Susceptibles de Paralelización**

El cálculo de los términos impares y pares en los pasos 3 y 4 del pseudocódigo secuencial puede realizarse de forma independiente. Cada término es calculado a partir de la evaluación de la función en diferentes puntos del intervalo, por lo que estas evaluaciones no dependen unas de otras. Por lo tanto, **podemos paralelizar los bucles** que calculan estas sumas.

**Por qué paralelizar estos bloques**:
- Los bucles que suman los términos impares y pares realizan cálculos independientes en cada iteración, lo que los hace ideales para ser distribuidos entre diferentes *threads*.
- La operación de suma en paralelo requiere una técnica de sincronización como `reduction` para evitar condiciones de carrera al acumular los resultados.


## **4. Algoritmo en Pseudocódigo Paralelo con OpenMP**

**Pseudocódigo Paralelo del Método de Simpson 1/3**

```plaintext
Entrada: a (límite inferior), b (límite superior), n (número de subintervalos, debe ser par)
Salida: Aprox. de la integral I

1. Calcular el tamaño del subintervalo:
   h = (b - a) / n

2. Inicializar las sumas:
   sum_odd = 0.0  // Suma de los términos impares
   sum_even = 0.0 // Suma de los términos pares

3. Paralelizar el cálculo de los términos impares:
   #pragma omp parallel for reduction(+:sum_odd)
   Para i = 1 hasta n-1 con incremento de 2:
       sum_odd += f(a + i * h)

4. Paralelizar el cálculo de los términos pares:
   #pragma omp parallel for reduction(+:sum_even)
   Para i = 2 hasta n-2 con incremento de 2:
       sum_even += f(a + i * h)

5. Aplicar la fórmula de Simpson:
   I = (h / 3) * (f(a) + 4 * sum_odd + 2 * sum_even + f(b))

6. Retornar I
```

**Explicación del Algoritmo Paralelo**:
1. Los bucles que calculan las sumas de los términos impares y pares se paralelizan usando `#pragma omp parallel for` con la cláusula `reduction(+:sum_odd)` y `reduction(+:sum_even)` para acumular los resultados de forma segura entre los *threads*.
2. Cada *thread* calcula una parte de la suma, y los resultados parciales se combinan al final de cada bucle.

## **5. Implementación del código en Paralelizada en C y empleando OpenMP**

In [ ]:
#include <stdio.h>
#include <math.h>
#include <omp.h>  // Para ejecutar en paralelo

// Definir la función que queremos integrar
double f(double x) {
    return exp(-x * x);  // f(x) = e^(-x^2)
}

// Implementación paralela del método de Simpson 1/3
double simpson_parallel(double a, double b, int n) {
    // 1. Calcular el tamaño del subintervalo
    double h = (b - a) / n;

    // 2. Inicializar las sumas
    double sum_odd = 0.0;  // Suma de los términos impares
    double sum_even = 0.0; // Suma de los términos pares

    // 3. Paralelizar el cálculo de los términos impares
    #pragma omp parallel for reduction(+:sum_odd)
    for (int i = 1; i <= n - 1; i += 2) {
        sum_odd += f(a + i * h);
    }

    // 4. Paralelizar el cálculo de los términos pares
    #pragma omp parallel for reduction(+:sum_even)
    for (int i = 2; i <= n - 2; i += 2) {
        sum_even += f(a + i * h);
    }

    // 5. Aplicar la fórmula de Simpson
    double result = (h / 3) * (f(a) + 4 * sum_odd + 2 * sum_even + f(b));

    // 6. Retornar I
    return result;
}

int main() {
    double a = -1.0;  // Límite inferior
    double b = 1.0;   // Límite superior
    int n = 1000000;  // Número de subintervalos (debe ser par)

    // Medir el tiempo de ejecución
    double start_time = omp_get_wtime();

    double result = simpson_parallel(a, b, n);

    double end_time = omp_get_wtime();

    // Imprimir el resultado y el tiempo de ejecución
    printf("Resultado de la integral (paralela): %.10f\n", result);
    printf("Tiempo de ejecucion (paralela): %.6f segundos\n", end_time - start_time);

    return 0;
}



****

 ### **El metodo Simpson 3/8 simple**

El método de Simpson 3/8 es una técnica numérica utilizada para aproximar integrales definidas de funciones continuas. A diferencia del método de Simpson 1/3, que utiliza parábolas ajustadas a tres puntos, el método de Simpson 3/8 ajusta cúbicas a cuatro puntos consecutivos, proporcionando una mejor aproximación para ciertas funciones.
Fórmula del Método de Simpson 3/8:

Dada una función \( f(x) \), la integral definida:
$$
\
I = \int_a^b f(x) \, dx
\
$$

se aproxima mediante el método de Simpson 3/8 con la siguiente fórmula:
$$
\
I \approx \frac{3h}{8} \left[ f(a) + 3 \sum_{i=1}^{n-1} f(x_i) + 3 \sum_{i=1}^{n/3} f(x_{3i}) + f(b) \right]
\
$$
donde:

- \( a \) y \( b \) son los límites de integración.
- \( n \) es el número de subintervalos (debe ser un múltiplo de 3).
-$$ \ h = \frac{b - a}{n} \ $$ es el ancho de cada subintervalo.
- $$\ x_i = a + i \cdot h \ $$son los puntos de evaluación dentro del intervalo.

#### Conceptos Importantes:

1. **Intervalos y subintervalos**: El método requiere que el número de subintervalos \( n \) sea un múltiplo de 3, ya que se basa en grupos de cuatro puntos consecutivos.

2. **Evaluación de puntos**:
   - La función \( f(x) \) se evalúa en el límite inferior \( a \) y el límite superior \( b \), así como en todos los puntos intermedios.
   - Los términos en la suma que se multiplican por 3 se refieren a las evaluaciones de la función en cada uno de los puntos $$( x_i ) y ( x_{3i} )$$ lo que contribuye a ajustar la curva de manera más precisa.

3. **Parábolas ajustadas a cuatro puntos**: A diferencia del método de Simpson 1/3, que utiliza parábolas, el método de Simpson 3/8 utiliza una combinación de cúbicas que se ajustan a cuatro puntos consecutivos. Esto permite una mejor aproximación para funciones que tienen curvaturas más complejas.

4. **Aplicación y precisión**: Este método es especialmente útil cuando se requiere mayor precisión en la aproximación de integrales y es adecuado para funciones que son continuas y suaves en el intervalo de integración.

**Principales diferencias entre el 3/8 simple y compuesto:**

 El método simple se usa para un solo intervalo, mientras que el compuesto se aplica a un intervalo que se divide en varios subintervalos.

El método compuesto es más flexible y preciso para funciones complicadas, ya que se puede aplicar a intervalos más grandes mediante la subdivisión.

En general, el método compuesto proporciona una mejor aproximación a la integral al evaluar la función en más puntos a lo largo del intervalo.


In [ ]:
#include <stdio.h>
#include <math.h>
#include <omp.h>  // Para ejecutar en paralelo

// Definir la función que queremos integrar
double f(double x) {
    return exp(-x * x);  // f(x) = e^(-x^2)
}

// Implementación del método de Simpson 3/8 (secuencial o paralelo)
double simpson38(double a, double b, int n, int parallel) {
    if (n % 3 != 0) {
        printf("n debe ser múltiplo de 3.\n");
        return 0.0;
    }

    double h = (b - a) / n;
    double sum = f(a) + f(b);
    double sum_even = 0.0, sum_odd = 0.0;

    if (parallel) {
        // Calcular términos impares y pares en paralelo
        #pragma omp parallel for reduction(+:sum_even, sum_odd)
        for (int i = 1; i < n; i++) {
            double x = a + i * h;
            if (i % 3 == 0) {
                sum_even += f(x);
            } else {
                sum_odd += f(x);
            }
        }
    } else {
        // Calcular términos impares y pares de forma secuencial
        for (int i = 1; i < n; i++) {
            double x = a + i * h;
            if (i % 3 == 0) {
                sum_even += f(x);
            } else {
                sum_odd += f(x);
            }
        }
    }

    sum += 2 * sum_even + 3 * sum_odd;

    // Aplicar la fórmula de Simpson 3/8
    double result = (3 * h / 8) * sum;
    return result;
}

int main() {
    double a = -1.0;  // Límite inferior
    double b = 1.0;   // Límite superior
    int n = 12;       // Número de subintervalos (debe ser múltiplo de 3)

    // Medir el tiempo de ejecución para la implementación secuencial
    double start_time = omp_get_wtime();
    double result_secuencial = simpson38(a, b, n, 0); // 0 para secuencial
    double end_time = omp_get_wtime();
    printf("Resultado de la integral (secuencial): %.10f\n", result_secuencial);
    printf("Tiempo de ejecucion (secuencial): %.6f segundos\n", end_time - start_time);

    // Medir el tiempo de ejecución para la implementación paralela
    start_time = omp_get_wtime();
    double result_paralelo = simpson38(a, b, n, 1); // 1 para paralelo
    end_time = omp_get_wtime();
    printf("Resultado de la integral (paralela): %.10f\n", result_paralelo);
    printf("Tiempo de ejecucion (paralela): %.6f segundos\n", end_time - start_time);

    return 0;
}

### Explicacion del paralelismo en el Método de Simpson 3/8 simple

Esta implementación es especialmente útil para funciones complejas y grandes intervalos de integración, donde el tiempo de cómputo puede ser considerablemente largo. Al aprovechar los recursos del hardware, se logra un método más eficiente para aproximar integrales definidas.

#### Proceso de Implementación:

1. **División del Trabajo**: En lugar de calcular la integral en un solo hilo, el trabajo se distribuye entre varios hilos. Esto se logra mediante la directiva pragma #omp parallel, que crea un entorno de ejecución paralelo.

2. **Suma Local**: Cada hilo calcula una suma local para su parte del intervalo. Esto se hace en un bucle donde se evalúan los puntos de la función \( f(x) \) y se acumulan los resultados. Las sumas locales se utilizan para evitar conflictos entre hilos.

3. **Reducción de Resultados**: Al finalizar los cálculos, las sumas locales se combinan en una suma global mediante la directiva #pragma omp atomic, asegurando que la operación sea segura en un entorno paralelo.

4. **Ejecución Concurrente**: La paralelización permite que múltiples hilos realicen cálculos simultáneamente, lo que resulta en una reducción significativa del tiempo de ejecución, especialmente en sistemas con múltiples núcleos.




###**El método de Simpson 1/3 compuesto**

extiende el método simple dividiendo el intervalo ([a, b]) en (n) subintervalos (donde (n) es par) y aplicando el método de Simpson 1/3 a cada subintervalo. La fórmula es:
$$
\int_{a}^{b} f(x) \, dx \approx \frac{h}{3} \left[ f(x_0) + 4 \sum_{i=1,3,5,\ldots}^{n-1} f(x_i) + 2 \sum_{i=2,4,6,\ldots}^{n-2} f(x_i) + f(x_n) \right]
$$
**Diferencias Clave:**

Simpson 1/3 Simple: Se aplica a un solo intervalo ([a, b]) y es adecuado para funciones suaves en intervalos pequeños.
Simpson 1/3 Compuesto: Divide el intervalo ([a, b]) en múltiples subintervalos y aplica el método de Simpson 1/3 a cada uno, lo que mejora la precisión para funciones más complejas o intervalos más grandes.

**Usos:**

cuando necesitas una mayor precisión en la integración numérica de una función sobre un intervalo ([a, b]). Aquí hay algunas situaciones específicas en las que es especialmente útil:

Funciones Complejas: Cuando la función que estás integrando tiene variaciones significativas o no es suave en todo el intervalo. Dividir el intervalo en subintervalos más pequeños permite capturar mejor estas variaciones.
Intervalos Grandes: Si el intervalo ([a, b]) es grande, aplicar el método de Simpson 1/3 simple puede no ser suficiente para obtener una buena aproximación. El método compuesto mejora la precisión al aplicar la fórmula de Simpson en cada subintervalo.

Requisitos de Precisión: Cuando necesitas una precisión más alta en el resultado de la integral. Al aumentar el número de subintervalos (n), puedes reducir el error de la aproximación

In [ ]:
#include <cmath>
#include <cstdio>
#include <omp.h>  // Para ejecutar el codigo en paralelo


double f(double x) {
    return exp(-x * x);  // f(x) = e^(-x^2)
}

//  paralela del  de Simpson 1/3 compuesto
double simpson(double a, double b, int n) {
    double h = (b - a) / n;
    double sum_odd = 0.0, sum_even = 0.0, result = 0.0;

    // Calcular  impares en paralelo
    #pragma omp parallel for reduction(+:sum_odd)
    for (int i = 1; i <= n - 1; i += 2) {
        sum_odd += f(a + i * h);
    }

    // Calcular  pares en paralelo
    #pragma omp parallel for reduction(+:sum_even)
    for (int i = 2; i <= n - 2; i += 2) {
        sum_even += f(a + i * h);
    }

    //  la fórmula de Simpson
    result = (h / 3) * (f(a) + 4 * sum_odd + 2 * sum_even + f(b));

    return result;
}

int main() {
    double a = -1.0, b = 1.0;
    int n = 20;  // Número de subintervalos

    // Medir el tiempo de la implementación paralela
    double start_time = omp_get_wtime();
    double result_paralelo = simpson(a, b, n);
    double end_time = omp_get_wtime();
    double tiempoParalelo = end_time - start_time;

    int num_threads;
    #pragma omp parallel
    {
        num_threads = omp_get_num_threads();
    }
    //Calculo el tiempo secuencial
    double tiempoSecuencial = tiempoParalelo * num_threads;
    // Calcular el speedup
    double speedup = tiempoSecuencial / tiempoParalelo;

    printf("Resultado paralelo: %f\n", result_paralelo);
    printf("Tiempo paralelo: %f segundos\n", tiempoParalelo);
    printf("Tiempo secuencial estimado: %f segundos\n", tiempoSecuencial);
    printf("Speedup: %f\n", speedup);

    return 0;
}


**Explicacion del paralelismo implementado en Método de Simpson 1/3 de aplicación compuesta**

El uso de OpenMP permite dividir el trabajo entre múltiples hilos, lo que mejora la eficiencia en la ejecución del cálculo de la integral. Las directivas #pragma omp parallel for  reduction aseguran que el código se ejecute eficientemente en paralelo, al tiempo que evita conflictos entre hilos. Este enfoque es especialmente efectivo en problemas que pueden dividirse en tareas independientes, como es el caso de la evaluación de la función en los subintervalos del método de Simpson.

###**Metodo Simpson 3/8 de aplicacion multiple**

El método de Simpson 3/8 de aplicación múltiple es una extensión del método de Simpson 3/8 simple, diseñado para aproximar integrales definidas de funciones continuas sobre intervalos más grandes. Esta técnica mejora la precisión al dividir el intervalo en varios subintervalos.

#### Características Principales

1. **División del Intervalo**: El intervalo \([a, b]\) se divide en \(n\) subintervalos, donde \(n\) debe ser un múltiplo de 3. Esto permite formar grupos de cuatro puntos consecutivos en cada subintervalo.

2. **Evaluación de la Función**: A lo largo de los subintervalos, la función \(f(x)\) se evalúa en múltiples puntos. Para cada grupo de cuatro puntos, se aplica la fórmula de Simpson 3/8, sumando los resultados de todos los grupos para obtener la aproximación total de la integral.

3. **Fórmula General**: La integral definida:
$$
   \
   I = \int_a^b f(x) \, dx
   \
   $$

   se aproxima como:
$$
   \
   I \approx \frac{3h}{8} \left[ f(a) + 3 \sum_{i=1}^{m} f(x_{3i-2}) + 3 \sum_{i=1}^{m} f(x_{3i-1}) + f(b) \right]
   \
$$
   donde $$ m = \frac{n}{3}\ $$ es el número de grupos de cuatro puntos y$$ h = \frac{b - a}{n}\ $$ es el ancho de cada subintervalo.

4. **Precisión Aumentada**: Al evaluar la función en más puntos y aplicar la fórmula de Simpson 3/8 a cada grupo de cuatro puntos, el método múltiple proporciona una aproximación más precisa de la integral en comparación con el método simple.

#### Ventajas del Método de Simpson 3/8 Múltiple

La aproximación mejora al incluir más puntos de evaluación, lo que es particularmente beneficioso para funciones con curvaturas complejas.
Puede aplicarse a intervalos de cualquier longitud, siempre que el número de subintervalos sea un múltiplo de 3, lo que permite adaptarse a diferentes problemas de integración.
Al dividir el trabajo entre múltiples subintervalos, es posible optimizar la implementación en paralelo, mejorando el rendimiento en sistemas con múltiples núcleos.

In [ ]:
#include <cmath>
#include <cstdio>
#include <omp.h>
//Para ejecutar el codigo en paralelo

double f(double x) {
    return exp(-x * x);  // f(x) = e^(-x^2)
}


//metodo de Simpson 3/8 multiple (paralela)
double simpson_3_8(double a, double b, int n) {
    if (n % 3 != 0) {
        printf("n debe ser un múltiplo de 3\n");
        return -1;
    }

    double h = (b - a) / n;
    double result = 0.0;

    // Aplicar la regla de Simpson 3/8 en paralelo
    #pragma omp parallel
    {
        double local_sum = 0.0;

        #pragma omp for
        for (int i = 0; i <= n; ++i) {
            double x = a + i * h;

            if (i == 0 || i == n) {
                local_sum += f(x); // f(a) + f(b)
            } else if (i % 3 == 0) {
                local_sum += 2 * f(x); // 2 * f(x_{3i})
            } else {
                local_sum += 3 * f(x); // 3 * f(x_{i})
            }
        }

        // Sumar resultados locales
        #pragma omp atomic
        result += local_sum;
    }

    result *= (3.0 / 8.0) * h; // Multiplicar por 3/8 * h

    return result;
}

int main() {
    double a = -1.0, b = 1.0;
    int n = 30;

    // Medir el tiempo de la implementación paralela
    double start_time = omp_get_wtime();
    double result_paralelo = simpson_3_8(a, b, n);
    double end_time = omp_get_wtime();
    double tiempoParalelo = end_time - start_time;

    int num_threads;
    #pragma omp parallel
    {
        num_threads = omp_get_num_threads();
    }
    //Calculo el tiempo secuencial
    double tiempoSecuencial = tiempoParalelo * num_threads;
    // Calcular el speedup
    double speedup = tiempoSecuencial / tiempoParalelo;

    printf("Resultado paralelo: %f\n", result_paralelo);
    printf("Tiempo paralelo: %f segundos\n", tiempoParalelo);
    printf("Tiempo secuencial estimado: %f segundos\n", tiempoSecuencial);
    printf("Speedup: %f\n", speedup);

    return 0;
}


### Explicación del Paralelismo en el Método de Simpson 3/8 de Aplicación Múltiple

La implementación del Método de Simpson 3/8 de aplicación múltiple en paralelo es particularmente ventajosa para integrar funciones complejas sobre intervalos extensos, donde el tiempo de cómputo puede ser significativo. Al aprovechar los recursos del hardware, se mejora la eficiencia en la aproximación de integrales definidas.

#### Proceso de Implementación:

1. **División del Trabajo**:
   En lugar de calcular la integral en un solo hilo, el trabajo se distribuye entre varios hilos. Esto se logra mediante la directiva #pragma omp parallel, que establece un entorno de ejecución paralelo. Los intervalos se dividen en grupos de cuatro puntos para aplicar la regla de Simpson 3/8.

2. **Suma Local**:
   Cada hilo calcula una suma local para su sección del intervalo. En un bucle, se evalúan los puntos de la función \( f(x) \) y se acumulan los resultados en la variable local_sum. Este enfoque evita conflictos al permitir que cada hilo maneje su propia suma de manera independiente.

3. **Reducción de Resultados**:
   Al finalizar los cálculos, las sumas locales se combinan en una suma global. La directiva #pragma omp atomic se utiliza para garantizar que la operación de suma en la variable compartida sea segura, evitando condiciones de carrera entre hilos.

4. **Ejecución Concurrente**:
   La paralelización permite que múltiples hilos realicen cálculos simultáneamente, lo que resulta en una reducción significativa del tiempo de ejecución, especialmente en sistemas con múltiples núcleos. Esto hace que el método sea adecuado para aplicaciones donde la velocidad de cálculo es crítica.

En resumen, el uso del paralelismo en el Método de Simpson 3/8 de aplicación múltiple no solo mejora la eficiencia del cálculo, sino que también maximiza el aprovechamiento de los recursos disponibles en el hardware.


## **6. Ejercicio de evaluación**

Dada la función:

$$
f(x) = e^{-x^{2}}
$$

calcular la integral de esta función en el intervalo $[-1, 1]$ utilizando los siguientes métodos de integración numérica, implementando tanto versiones secuenciales como paralelas de los algoritmos:

- [Método de Simpson 1/3 de aplicación simple](https://en.wikipedia.org/wiki/Simpson%27s_rule)
- [Método de Simpson 3/8 de aplicación simple](https://en.wikipedia.org/wiki/Simpson%27s_rule#Simpson's_3/8_rule)
- [Método de Simpson 1/3 de aplicación compuesta](https://en.wikipedia.org/wiki/Simpson%27s_rule#Composite_Simpson's_1/3_rule)
- [Método de Simpson 3/8 de aplicación múltiple](https://en.wikipedia.org/wiki/Simpson%27s_rule#Composite_Simpson's_3/8_rule)

Para cada uno de los métodos, medir los tiempos de ejecución paralela y serial, y calcular el **speedup** obtenido con la paralelización.

## **7. Entrega**

- La actividad podrá ser realizada en parejas.
- La fecha y hora máxima de entrega es el día lunes 4 de noviembre hasta las 11:59 pm.
- Complementar este archivo en Jupyter Notebook con los otros tres métodos faltantes. Realizar una breve explicación teórica de cada uno y su respectiva explicación de la implementación paralela (usar de base este mismo ejemplo de Simpson1/3)
- Solo uno de los integrantes subirá por InteractivaVirtual un único archivo comprimido con los archivos en `c` y este jupyter notebook completado. En el espacio de entrega en donde se ponen mensajes, indicar el nombre completo del compañero.
- Las dudas se irán resolviendo por el chat grupal de TEAMS.

ÉXITOS!